In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv("../input/heart-disease-classification-dataset-cse422-brac/heart disease classification dataset.csv")
data.head()

In [ ]:
data.drop("Unnamed: 0", axis=1, inplace=True)
data.head()

In [ ]:
data.isnull().sum()

In [ ]:
data.dropna(inplace=True)

In [ ]:
from sklearn.preprocessing import LabelEncoder

LE = LabelEncoder()
encoded = LE.fit_transform(data.iloc[:, -1])
Y = pd.DataFrame(encoded)
# Y.rename(columns='Drug', inplace=True)
Y.head()

In [ ]:
X = data.iloc[:, :-1]
X.head()

In [ ]:
cat_cols = X.dtypes == object
cat_cols = X.columns[cat_cols].to_list()
cat_cols

In [ ]:
num_cols = X.dtypes != object
num_cols = X.columns[num_cols].to_list()
num_cols

In [ ]:
LE2 = LabelEncoder()
cat_encoded = LE2.fit_transform(X[cat_cols])
X.loc[:, cat_cols] = cat_encoded
X.head()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.3, random_state=42)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

classifier = DecisionTreeClassifier()
classsifier = classifier.fit(X_train, Y_train)
prediction = classifier.predict(X_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

RFclassifier = RandomForestClassifier()
RFclassifier = RFclassifier.fit(X_train, Y_train)
RFprediction = RFclassifier.predict(X_test)

In [ ]:
from sklearn.metrics import plot_roc_curve

plot_roc_curve(classifier, X_test, Y_test)

In [ ]:
plot_roc_curve(RFclassifier, X_test, Y_test)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score

def evaluation_metrics1(Y_true, pred):
    tn, fp, fn, tp = confusion_matrix(Y_true, pred).ravel()
    acc = accuracy_score(Y_true, pred)
    pre = precision_score(Y_true, pred)
    re = recall_score(Y_true, pred)
    f1 =  f1_score(Y_true, pred)
    return tn, fp, fn, tp, acc, pre, re, f1

In [ ]:
tn1, fp1, fn1, tp1, acc1, pre1, re1, f1 = evaluation_metrics1(Y_test, prediction)

In [ ]:
tn2, fp2, fn2, tp2, acc2, pre2, re2, f2 = evaluation_metrics1(Y_test, RFprediction)

In [ ]:
comparision = pd.DataFrame([[tn1, fp1, fn1, tp1, acc1, pre1, re1, f1], [tn2, fp2, fn2, tp2, acc2, pre2, re2, f2]],
                           columns=["True Neg", "False Pos", "False Neg", "True Pos", "accuracy", "precision", "recall", "f1-score"],
                          index=["Decision Trees", "Random Forest"])
comparision

In [ ]:
# from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score

# def evaluation_metrics(Y_true, pred):
#     print("confusion matrix")
#     print(confusion_matrix(Y_true, pred))
#     print("accuracy score: ",accuracy_score(Y_true, pred))
#     print("precision score: ",precision_score(Y_true, pred))
#     print("recall score: ", recall_score(Y_true, pred))
#     print("F1 score: ", f1_score(Y_true, pred))

In [ ]:
# evaluation_metrics(Y_test, prediction)

In [ ]:
# evaluation_metrics(Y_test, RFprediction)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

trainScaler = MinMaxScaler()
xTrainScaled = trainScaler.fit_transform(X_train[num_cols])
X_train.loc[:, num_cols] = xTrainScaled
X_train.head()

In [ ]:
testScaler = MinMaxScaler()
xTestScaled = testScaler.fit_transform(X_test[num_cols])
X_test.loc[:, num_cols] = xTestScaled
X_test.head()

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn5 = KNeighborsClassifier(n_neighbors=5)
knn5_fit = knn5.fit(X_train, Y_train)
knn5_pred = knn5_fit.predict(X_test)
tn3, fp3, fn3, tp3, acc3, pre3, re3, f3 = evaluation_metrics1(Y_test, knn5_pred)

In [ ]:
plot_roc_curve(knn5_fit, X_test, Y_test)

In [ ]:
knn = pd.DataFrame([[tn3, fp3, fn3, tp3, acc3, pre3, re3, f3]],
                columns=["True Neg", "False Pos", "False Neg", "True Pos", "accuracy", "precision", "recall", "f1-score"],
                  index=['KNN'])

In [ ]:
comparision = pd.concat([comparision, knn])
comparision